In [1]:
import sympy as sp
import numpy as np

In [117]:
# basis vectors in x, y, and t
x=sp.Symbol('x')
y=sp.Symbol('y')
t=sp.Symbol('t')
#model and data amplitudes
Ad=sp.Symbol('A_d')
Am=sp.Symbol('A_m')

# domain width
L=sp.Symbol('L', positive=True, finite=True)
# domain time dinension
T=sp.Symbol('T', positive=True, finite=True)

# data errors:
s=sp.Symbol('\sigma')
# data density:
rd=sp.Symbol('rho', positive=True, finite=True)
# wavenumbers in the x, y, and t dimensions
kx=sp.Symbol('k_x', integer=True, positive=True)
ky=sp.Symbol('k_y', integer=True, positive=True)
kt=sp.Symbol('k_t', integer=True, positive=True)
# Weighting on d3z/dx2dt
Wxx=sp.Symbol('W_{xx}')
# Weighting on d2x/dt2
Wtt=sp.Symbol('W_{tt}')

In [194]:
# basis functions:
bf =sp.sin(2*kx*sp.pi*x/L) * sp.sin(2*ky*sp.pi*y/L) * sp.sin(2*kt*sp.pi*t/T) 
# model is the basis function times Am
m=Am * bf
# data is basis function times Ad
d=Ad * bf

In [195]:
# derivatives of the basis functions
D2xtm = sp.diff(sp.diff(sp.diff(m, x),x),t)
D2ytm = sp.diff(sp.diff(sp.diff(m, y),y),t)
Dxytm = sp.diff(sp.diff(sp.diff(m, x),y),t)
D2tm = sp.diff(sp.diff(m, t),t)

In [196]:
# Combined residual:
R=sp.integrate(\
               sp.integrate(\
    sp.integrate( rd*((d-m)/s)**2 + (Wxx*D2xtm)**2 + (Wxx*D2ytm)**2 + 2*(Wxx*Dxytm)**2+(Wtt*D2tm)**2, (x, 0, L)),\
    (y, 0, L)), 
               (t, 0, T))

In [197]:
# solve for model amplitude that minimizes the combined residual:
A_best=sp.solve(sp.diff(R, Am), Am)[0]
A_best

A_d*L**4*T**4*rho/(L**4*T**4*rho + 16*pi**4*L**4*W_{tt}**2*\sigma**2*k_t**4 + 64*pi**6*T**2*W_{xx}**2*\sigma**2*k_t**2*k_x**4 + 128*pi**6*T**2*W_{xx}**2*\sigma**2*k_t**2*k_x**2*k_y**2 + 64*pi**6*T**2*W_{xx}**2*\sigma**2*k_t**2*k_y**4)

In [198]:
# simplify the denominator:
sp.expand(1/(A_best/(Ad)))

1 + 16*pi**4*W_{tt}**2*\sigma**2*k_t**4/(T**4*rho) + 64*pi**6*W_{xx}**2*\sigma**2*k_t**2*k_x**4/(L**4*T**2*rho) + 128*pi**6*W_{xx}**2*\sigma**2*k_t**2*k_x**2*k_y**2/(L**4*T**2*rho) + 64*pi**6*W_{xx}**2*\sigma**2*k_t**2*k_y**4/(L**4*T**2*rho)

This expression is equivalent to A_best

In [200]:
sp.simplify(Ad*rd/s**2 /  (rd/s**2 + 16*sp.pi**4*(Wtt**2*kt**4/T**4 + 4*sp.pi**2*kt**2*Wxx**2*(kx**2+ky**2)**2/(L**4*T**2))))

A_d*L**4*T**4*rho/(L**4*T**4*rho + 16*pi**4*\sigma**2*k_t**2*(L**4*W_{tt}**2*k_t**2 + 4*pi**2*T**2*W_{xx}**2*(k_x**2 + k_y**2)**2))

If we want to look at the relationship between data bias errors and large-wavelength second derivatives, we look for the period where the second term in the denominator becomes equal to the first.  

To suppress oscillations of period  < 0.125 yr, for data errors of 0.1 m, with 1 data point /3 km:

In [213]:
tau=0.125  # 1/8 year period
Ti=1 # 1 year
kti = Ti/tau
rho_tr=4/3000   # 4 measurements/year/3 km
si=0.1 # 0.1 m error
Wtti = np.sqrt((Ti**4*rho_tr)/(16*np.pi**4*kti**4*si**2))
print(1/Wtti)

6919.4303540850005


In [216]:
tau=0.125
Ti=1
kti = Ti/tau
si=0.1
rho_dp=4*((2*3000/30)/3000/3000)   # 4 measurements/year*(*2 tracks*3km /60 m/pt)/(3 km x 3km)
Wtti = np.sqrt((Ti**4*rho_dp)/(16*np.pi**4*kti**4*si**2))
print(1/Wtti)

26798.838526613294


In [225]:
# if we want to suppress 1-km scale variations in dh/dt at a temporal scale of 0.25 yr:
Li=1000
wvl=500
Ti=1
tau=0.25
kti=Ti/tau
kxi=Li/wvl
Wxxi = Li**2*Ti/(8*np.pi**3*si*kti*kxi**2)
print(1/Wxxi)

0.0003968803415078377


In [226]:
! hostname

kjer
